In [1]:
!pip install pororo

     |████████████████████████████████| 256 kB 12.5 MB/s 
     |████████████████████████████████| 468 kB 32.2 MB/s 
     |████████████████████████████████| 42.4 MB 1.4 MB/s 
     |████████████████████████████████| 1.5 MB 20.7 MB/s 
     |████████████████████████████████| 1.2 MB 36.9 MB/s 
     |████████████████████████████████| 3.1 MB 38.6 MB/s 
     |████████████████████████████████| 79 kB 1.6 MB/s 
     |████████████████████████████████| 3.5 MB 13.4 MB/s 
     |████████████████████████████████| 748.8 MB 18 kB/s 
     |████████████████████████████████| 5.9 MB 22.2 MB/s 
     |████████████████████████████████| 1.7 MB 41.0 MB/s 
     |████████████████████████████████| 145 kB 51.3 MB/s 
     |████████████████████████████████| 90 kB 9.5 MB/s 
     |████████████████████████████████| 748 kB 45.7 MB/s 
     |████████████████████████████████| 1.2 MB 41.7 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 596 kB 34.6 MB/s 
     |██████████████

In [2]:
!pip install python-mecab-ko==1.0.9

     |████████████████████████████████| 211 kB 12.5 MB/s 
  ERROR: Failed building wheel for python-mecab-ko
  Running setup.py clean for python-mecab-ko
Failed to build python-mecab-ko
    Running setup.py install for python-mecab-ko ... done
  DEPRECATION: python-mecab-ko was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [3]:
!pip install transformers

In [4]:
from pororo import Pororo

In [5]:
#transformers
from transformers import AdamW,AutoTokenizer
from transformers.optimization import get_cosine_schedule_with_warmup # Lr 스케줄링, 보통 학습 초기엔 높은 Lr로 빠른 탐색 후, 학습 후기 낮은 Lr로 세부 탐색을 진행한다.

In [6]:
from google.colab import drive
drive.mount('/content/drive')
path='/content/drive/My Drive/Colab Notebooks/Data/Korean_Sentence_classification(Dacon)/' # 구글 클라우드 데이터 위치

Mounted at /content/drive


In [7]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [8]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [9]:
train_tmp=pd.read_csv(path+'train_data.csv').set_index('index')
test_tmp=pd.read_csv(path+'test_data.csv').set_index('index')
submission=pd.read_csv(path+'sample_submission.csv')

In [10]:
tok=AutoTokenizer.from_pretrained("klue/roberta-large")

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
le=LabelEncoder()
train_tmp['label']=le.fit_transform(train_tmp['label'])
x_test_tmp=test_tmp.iloc[:,:-1]

In [13]:
y_train_tmp=train_tmp['label']
x_train_tmp=train_tmp.iloc[:,:-1]

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train,x_val,y_train,y_val=train_test_split(x_train_tmp,y_train_tmp,test_size=0.2,stratify=y_train_tmp,shuffle=True,random_state=50)

In [16]:
x_train

,premise,hypothesis
index,,
20942,"세월호 특별법의 협상이 타결된 지 하루만에 인양이야기가 나왔고, 이틀째인 오늘은 협...",세월호 인양문제는 2일차에 거론되었습니다.
20787,괜찮은 주제와 스토리를 전혀 살리지 못하는 최악의 연기,괜찮은 것이라곤 연기 뿐이다.
20354,"이렇게 정당이 국가적 의의가 있는 중요한 공공의 기능을 가지고 있기는 하나, 그렇다...",정당은 중요한 공공 기능을 가지고 있다.
1625,남자는 두 소년의 가족을 죽이겠다고 협박하면서 경찰관이 오면 아무것도 말하지 말라고...,두 소년의 가족은 경찰에 의해 안전하게 있다.
11907,이 숙소는 정말로 너무 완벽하였습니다.,모든것이 마음에 드는 숙소였습니다.
...,...,...
6367,"이번 지역사랑상품권법 시행령의 주요 내용으로는, 첫째 건전한 유통질서 확립을 위해 ...",과태료는 행위에 따라 최소 100만원에서 최대 2000만원까지이다.
4986,22일 오전 서울 강남구 봉은사로에 위치한 바비레드 강남점에 인앤아웃 버거 팝업스토...,22일 오후부터 바비레드 강남점에 인앤아웃 버거 팝업스토어가 열렸다.
20775,"오늘 야당의 의원총회가 열렸고, 그 전에 여야의 원내대표가 세월호 유가족과 3자 회...",여야의 대표 국회의원은 야당 의원총회가 열린 다음 날에 세월호 유가족과 세 번째 만...


In [17]:
y_train

index
20942    0
20787    0
20354    1
1625     2
11907    1
        ..
6367     2
4986     0
20775    0
3132     0
12931    0
Name: label, Length: 19998, dtype: int64

In [18]:
x_train.reset_index(drop=True)

,premise,hypothesis
0,"세월호 특별법의 협상이 타결된 지 하루만에 인양이야기가 나왔고, 이틀째인 오늘은 협...",세월호 인양문제는 2일차에 거론되었습니다.
1,괜찮은 주제와 스토리를 전혀 살리지 못하는 최악의 연기,괜찮은 것이라곤 연기 뿐이다.
2,"이렇게 정당이 국가적 의의가 있는 중요한 공공의 기능을 가지고 있기는 하나, 그렇다...",정당은 중요한 공공 기능을 가지고 있다.
3,남자는 두 소년의 가족을 죽이겠다고 협박하면서 경찰관이 오면 아무것도 말하지 말라고...,두 소년의 가족은 경찰에 의해 안전하게 있다.
4,이 숙소는 정말로 너무 완벽하였습니다.,모든것이 마음에 드는 숙소였습니다.
...,...,...
19993,"이번 지역사랑상품권법 시행령의 주요 내용으로는, 첫째 건전한 유통질서 확립을 위해 ...",과태료는 행위에 따라 최소 100만원에서 최대 2000만원까지이다.
19994,22일 오전 서울 강남구 봉은사로에 위치한 바비레드 강남점에 인앤아웃 버거 팝업스토...,22일 오후부터 바비레드 강남점에 인앤아웃 버거 팝업스토어가 열렸다.
19995,"오늘 야당의 의원총회가 열렸고, 그 전에 여야의 원내대표가 세월호 유가족과 3자 회...",여야의 대표 국회의원은 야당 의원총회가 열린 다음 날에 세월호 유가족과 세 번째 만...
19996,미국 캘리포니아에 발생한 대형 화재를 진압하던 소방관들이 갑자기 느려진 데이터 속도...,갑자기 빨라진 데이터 속도로 소방관들의 구호 작업에 차질을 빚었다.


In [26]:
for_da=pd.concat([x_train.reset_index(drop=True),y_train.reset_index(drop=True)],axis=1)

In [27]:
for_da_1=for_da[:10000]
for_da_2=for_da[10000:]

In [19]:
pg=Pororo(task='pg',lang='ko')
gec=Pororo(task='gec',lang='ko')

As of now, this beta model tries to correct spacing errors in Korean text.


In [30]:
def for_DA(x,y):
  y=y.copy().reset_index(drop=True)
  x=np.array(x.copy().reset_index(drop=True))
  df=pd.DataFrame({'premise':[],'hypothesis':[],'label':[]})
  for i,(sen1,sen2) in enumerate(tqdm(x)):
    df.append({'premise':pg(gec(sen1)),'hypothesis':pg(gec(sen2)),'label':y[i]},ignore_index=True)
  return df

In [1]:
train_da_1=for_DA(for_da_1.iloc[:,:-1],for_da_1.iloc[:,-1])
train_da_1.to_excel(path+'train_da_1.xlsx')

NameError: ignored

In [ ]:
train_da_2=for_DA(for_da_2.iloc[:,:-1],for_da_2.iloc[:,-1])
train_da_2.to_excel(path+'train_da_1.xlsx')

In [ ]:
def ConvertX(x,tokenizer,max_len):
  sent1=x.iloc[:,0].reset_index(drop=True)
  sent2=x.iloc[:,1].reset_index(drop=True)
  input_ids=[]
  token_type_ids=[]
  attention_mask=[]
  for sen1,sen2 in tqdm(zip(sent1,sent2), total=len(sent1)):
    tmp=tokenizer.encode_plus(
        sen1,sen2,
        max_length=max_len,
        pad_to_max_length=True,
        #padding='max_length',
        )
    input_ids.append(tmp['input_ids'])
    token_type_ids.append(tmp['token_type_ids'])
    attention_mask.append(tmp['attention_mask'])
  input_ids=np.array(input_ids,dtype=int)
  token_type_ids=np.array(token_type_ids,dtype=int)
  attention_mask=np.array(attention_mask,dtype=int)
  return (input_ids,attention_mask,token_type_ids)
    

In [ ]:
# Setting parameters
max_len = 150 # 최대 문장 길이
batch_size = 32
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
learning_rate =  1e-5

In [ ]:
x_train_convert=ConvertX(x_train,tok,max_len)
x_val_convert=ConvertX(x_val,tok,max_len)
x_test_convert=ConvertX(x_test_tmp,tok,max_len)
y_train=y_train.reset_index(drop=True)
y_val=y_val.reset_index(drop=True)

  0%|          | 0/19998 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1666/1666 [00:00<00:00, 4384.64it/s]


In [ ]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [ ]:
from transformers import TFAutoModel
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow_addons.optimizers import AdamW
from tensorflow_addons.optimizers import RectifiedAdam

In [ ]:
class KLUEClassifier(Model):
    def __init__(self, model, hidden_size = 768, num_classes=3, dr_rate=0): #클래스 수 확인
        super(KLUEClassifier, self).__init__() # Classifier가 상속한 Model 인자를 Classifier가 가져온다.
        self.model = model.from_pretrained("klue/roberta-base",num_labels=num_classes,from_pt=True) # model: TFAutoModel
                  
        #tmp_classifier= Sequential()
        #tmp_classifier.add(layers.Dense(input_dim=hidden_size,units=num_classes))
        #tmp_classifier.add(layers.Activation('softmax'))
        #tmp_classifier.add(layers.Dropout(rate=dr_rate))
        #self.classifier=tmp_classifier
        self.classifier=layers.Dense(num_classes,kernel_initializer=TruncatedNormal(0.02),activation='softmax')
        # 단층 nn (입력 층 hidden_size, 출력층 num_classes)

    def call(self, inputs): # 모델과 함께 데이터를 입력하면 자동으로 실행되는 함수    
        Out = self.model(input_ids = inputs[0], attention_mask = inputs[1], token_type_ids = inputs[2]) # 모델 실행
        return self.classifier(Out[1])

In [ ]:
t_total = len(y_train)//batch_size * num_epochs
#warmup_step = int(t_total * warmup_ratio) # warmup 방식 -> 학습률을 점차 상승 시켰다가 정점을 지난 후 다시 감소시키는 방식

#step = tf.Variable(0, trainable=False)
#schedule = optimizers.schedules.CosineDecay(learning_rate,warmup_step)

#optimizer와 schedule 설정
#optimizer = AdamW(learning_rate=learning_rate, weight_decay=schedule)
optimizer = RectifiedAdam(learning_rate=learning_rate, total_steps=t_total, warmup_proportion=warmup_ratio,min_lr=1e-8) # Lr은 t_total에서 warmup_ratio 만큼 증가했다가 그 이후로는 감소한다.
loss = 'sparse_categorical_crossentropy' # 다중분류를 위한 손실함수 이 때 sparse는 원핫인코딩을 하지 않아도 되는 손실함수이다

In [ ]:
with tf.device('/device:GPU:0'): # 모델 학습에 있어 GPU 학습 사용
  Mod=KLUEClassifier(TFAutoModel,hidden_size=1024,dr_rate=0)
  Mod.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
  Mod.fit(
      x_train_convert,y_train,
      batch_size=batch_size, 
      epochs=num_epochs,   
      validation_data=(x_val_convert,y_val),
      callbacks=[EarlyStopping(monitor='val_accuracy',patience=2)]
      #verbose=1,
      ) #전 과정을 화면에 출력(1) or 미출력(0)

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.embeddings.position_ids', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream 

Epoch 1/20
625/625 [==============================] - 873s 1s/step - loss: 0.9312 - accuracy: 0.4977 - val_loss: 0.4375 - val_accuracy: 0.8354
Epoch 2/20
625/625 [==============================] - 850s 1s/step - loss: 0.4210 - accuracy: 0.8433 - val_loss: 0.3814 - val_accuracy: 0.8540
Epoch 3/20
625/625 [==============================] - 857s 1s/step - loss: 0.2905 - accuracy: 0.8949 - val_loss: 0.4335 - val_accuracy: 0.8520
Epoch 4/20
625/625 [==============================] - 859s 1s/step - loss: 0.1836 - accuracy: 0.9352 - val_loss: 0.4523 - val_accuracy: 0.8650
Epoch 5/20
625/625 [==============================] - 861s 1s/step - loss: 0.1253 - accuracy: 0.9566 - val_loss: 0.4641 - val_accuracy: 0.8690
Epoch 6/20
625/625 [==============================] - 813s 1s/step - loss: 0.0877 - accuracy: 0.9697 - val_loss: 0.5638 - val_accuracy: 0.8644
Epoch 7/20
625/625 [==============================] - 836s 1s/step - loss: 0.0604 - accuracy: 0.9782 - val_loss: 0.4922 - val_accuracy: 0.8722

In [ ]:
preds_tmp=Mod.predict(x_test_convert)
preds=[np.argmax(pred) for pred in preds_tmp]

In [ ]:
label_idx = dict(zip( le.transform(list(le.classes_)),list(le.classes_)))
label_idx

{0: 'contradiction', 1: 'entailment', 2: 'neutral'}

In [ ]:
preds_decode=[label_idx[pred] for pred in preds]

In [ ]:
submission['label'] = preds_decode
submission.to_csv(path+"submission.csv", index = False)